# Instalar dependencias

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install accelerate>=0.12.0 datasets>=1.8.0 sentencepiece!=0.1.92 scipy scikit-learn protobuf torch>=1.3 transformers evaluate


q

ERROR: Operation cancelled by user


# Usando modelos pre-entrenados de HuggingFace
En esta section vamos a probar modelos disponibles en [HuggingFace](https://https://huggingface.co/), un repositorio masivo de modelos y datasets para diversas aplicaciones de NLP, Vision, Audio, entre otros.

Como libreria backbone emplearemos Transformers, que a su vez extiende pytorch.

Modelo a usar: Qwen2.0-0.5 Instruct
* basado en Qwen, una modelo opensource de la misma arquitectura que GPT-3 [link text](https://qwenlm.github.io/blog/qwen2/)
* entrenado con texto en 27 lenguajes

In [3]:
from transformers import pipeline, set_seed

# generator = pipeline('text-generation', model='nlpcloud/instruct-gpt-j-fp16')
generator = pipeline('text-generation', model='Qwen/Qwen2-0.5B-Instruct')

set_seed(42)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
output = generator("Explicame como se mueve la tierra alrededor del sol", max_length=200)
print(output[0]["generated_text"])

Explicame como se mueve la tierra alrededor del sol. La Tierra y el Sol están en un ángulo de aproximadamente 90 grados, lo que significa que cada día, el Sol pasa por un rango de 180 grados, pero el movimiento de la Tierra es hacia el Sol a medida que hace una circunferencia de aproximadamente 360 grados.

Por lo tanto, si consideramos que cada día la Tierra sale de la Luna a 24 horas, entonces la Tierra está moviendo hacia el Sol a medida que hace 360 grados (360/24 = 15) días.

Este movimiento constante de la Tierra hacia el Sol no es completamente impredecible debido a la influencia de los planetas celestes, incluyendo la estrella Neptúnico y la estrella Júpiter, pero generalmente


# Tecnicas de Prompting

## Zero-shot

In [4]:
output = generator("""
Descripcion: Clasifica el texto en Positivo o Negativo
Texto: Me encanto el producto.
Respuesta:
""", max_new_tokens=4, temperature=0.7)
print(output[0]["generated_text"])


Descripcion: Clasifica el texto en Positivo o Negativo
Texto: Me encanto el producto.
Respuesta:
Positiva




In [ ]:
output = generator("""
Descripcion: Clasifica el texto en Positivo o Negativo
Texto: Terrible producto, una perdidad de dinero.
Respuesta:
""", max_new_tokens=4, temperature=0.7)
print(output[0]["generated_text"])


Descripcion: Clasifica el texto en Positivo o Negativo
Texto: Terrible producto, una perdidad de dinero.
Respuesta:
Positiva




## Few-shot

In [8]:
output = generator("""
Descripcion: Clasifica el texto en Positivo o Negativo
Texto: Buen precio y buen servicio.
Respuesta: Positivo

Texto: Muy caro y mala comida.
Respuesta: Negativo

Texto: El ambiente es agradable y la comida es deliciosa.
Respuesta:""", max_new_tokens=50, temperature=0.7)
print(output[0]["generated_text"])


Descripcion: Clasifica el texto en Positivo o Negativo
Texto: Buen precio y buen servicio.
Respuesta: Positivo

Texto: Muy caro y mala comida.
Respuesta: Negativo

Texto: El ambiente es agradable y la comida es deliciosa.
Respuesta: Positivo

Texto: La calidad de los productos es excelente, pero el precio es un poco alto.
Respuesta: Positivo

Texto: El servicio fue genial y el desayuno era muy bueno.
Respuesta: Posit


In [7]:
output[0]['generated_text']

'\nDescripcion: Clasifica el texto en Positivo o Negativo\nTexto: Buen precio y buen servicio.\nRespuesta: Positivo\n\nTexto: Muy caro y mala comida.\nRespuesta: Negativo\n\nTexto: El ambiente es agradable y la comida es deliciosa.\nRespuesta: Positivo\n\nPor favor, proporciona un'

In [9]:
output = generator("""
Descripcion: Clasifica el texto en Positivo o Negativo
Texto: Buen precio y buen servicio.
Respuesta: Positivo

Texto: Muy caro y mala comida.
Respuesta: Negativo

Texto: El ambiente es desagradable y la comida es mala.
Respuesta:""", max_new_tokens=10, temperature=0.7)
print(output[0]["generated_text"])


Descripcion: Clasifica el texto en Positivo o Negativo
Texto: Buen precio y buen servicio.
Respuesta: Positivo

Texto: Muy caro y mala comida.
Respuesta: Negativo

Texto: El ambiente es desagradable y la comida es mala.
Respuesta: Negativo

Por favor, proporciona un texto


## Chain of Thought

La idea principal es pedirle al modelo que explique su razonamiento en pasos.

In [10]:
output = generator("""
Q: Roger tiene 5 pelotas. El compra 2 cajas de pelotas. Cada caja tiene 3 pelotas. ¿Cuántos pelotas tiene Roger?
A: La respuesta es 11.

Q: La cafeteria tenia 23 manzanas. Si ellos usaron 20 para preparar el almuerzo y luego compraron 6 manzanas mas. ¿Cuántas manzanas tiene la cafeteria?
A:""", max_new_tokens=10, temperature=0.7)
print(output[0]["generated_text"])


Q: Roger tiene 5 pelotas. El compra 2 cajas de pelotas. Cada caja tiene 3 pelotas. ¿Cuántos pelotas tiene Roger?
A: La respuesta es 11.

Q: La cafeteria tenia 23 manzanas. Si ellos usaron 20 para preparar el almuerzo y luego compraron 6 manzanas mas. ¿Cuántas manzanas tiene la cafeteria?
A: La respuesta es 47.

Q: Un


In [12]:
output = generator("""
Q: Roger tiene 5 pelotas. El compra 2 cajas de pelotas. Cada caja tiene 3 pelotas. ¿Cuántos pelotas tiene Roger?
A: Roger empezo con 5 pelotas. 2 cajas de 3 pelotas cada una son 6 pelotas. 5 + 6 = 11. La respuesta es 11.

Q: La cafeteria tenia 23 manzanas. Si ellos usaron 20 para preparar el almuerzo y luego compraron 6 manzanas mas. ¿Cuántas manzanas tiene la cafeteria?
A:""", max_new_tokens=100, temperature=0.7)
print(output[0]["generated_text"])


Q: Roger tiene 5 pelotas. El compra 2 cajas de pelotas. Cada caja tiene 3 pelotas. ¿Cuántos pelotas tiene Roger?
A: Roger empezo con 5 pelotas. 2 cajas de 3 pelotas cada una son 6 pelotas. 5 + 6 = 11. La respuesta es 11.

Q: La cafeteria tenia 23 manzanas. Si ellos usaron 20 para preparar el almuerzo y luego compraron 6 manzanas mas. ¿Cuántas manzanas tiene la cafeteria?
A: La cafeteria tenía 23 - 20 = 3 manzanas después de que se los compró más. De esta manera, la cafeteria tiene 3 + 6 = 9 manzanas en total. La respuesta es 9.

Q: Una empresa produce 100 piezas por día. ¿Cómo se divide entre 4 días?

A: Para dividir entre 4 días, primero debemos encontrar el número total de piezas producidas por una


In [13]:
output = generator("""
Q: Roger tiene 5 pelotas. El compra 2 cajas de pelotas. Cada caja tiene 3 pelotas. ¿Cuántos pelotas tiene Roger?
A: Step by Step in JSON format
{
  'step 1': 'Roger empezo con 5 pelotas',
  'step 2': '2 cajas de 3 pelotas cada una son 6 pelotas',
  'step 3': '5 + 6 = 11',
  'answer': 11
}

Q: La cafeteria tenia 23 manzanas. Si ellos usaron 20 para preparar el almuerzo y luego compraron 6 manzanas mas. ¿Cuántas manzanas tiene la cafeteria?
A:""", max_new_tokens=200, temperature=0.7)
print(output[0]["generated_text"])


Q: Roger tiene 5 pelotas. El compra 2 cajas de pelotas. Cada caja tiene 3 pelotas. ¿Cuántos pelotas tiene Roger?
A: Step by Step in JSON format
{
  'step 1': 'Roger empezo con 5 pelotas',
  'step 2': '2 cajas de 3 pelotas cada una son 6 pelotas',
  'step 3': '5 + 6 = 11',
  'answer': 11
}

Q: La cafeteria tenia 23 manzanas. Si ellos usaron 20 para preparar el almuerzo y luego compraron 6 manzanas mas. ¿Cuántas manzanas tiene la cafeteria?
A: Step by Step in JSON format
{
 'step 1': 'La cafeteria tenía 23 manzanas',
 'step 2': 'Usaron 20 manzanas',
 'step 3': 'Luego compró 6 manzanas más',
  'answer': 47
} 

Q: Un libro puede ser considerado como un conjunto de páginas separados. Por lo tanto, si hay 396 páginas en un libro, ¿cuál es el número total de páginas que el libro contiene?

A: Step by Step in JSON format
{
 'step 1': 'Un libro puede contener múltiples páginas.',
 'step 2': 'Hay 396 páginas en un libro.',
  'answer': 'El número total de páginas del libro es 396.'
} 

Q: A empr